In [10]:
import tkinter as tk
from tkinter import messagebox
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity


In [11]:
# Carregar o dataset de filmes
movies = pd.read_csv("datasets/movies_metadata.csv")
movies['title'].head()
# movies['genres'].head()


C:\Users\rondi\AppData\Local\Temp\ipykernel_8096\3684815186.py:2: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  movies = pd.read_csv("datasets/movies_metadata.csv")


0                      Toy Story
1                        Jumanji
2               Grumpier Old Men
3              Waiting to Exhale
4    Father of the Bride Part II
Name: title, dtype: object

In [12]:
#pre processando os generos e criando a matriz de simmilaridade

def create_similarity_model():
    # Combina título e gêneros
    movies['combined_features'] = movies['title'] + ' ' + movies['genres'].fillna('')
    
    # Vetorizar os dados usando TF-IDF
    tfidf = TfidfVectorizer(stop_words="english")
    tfidf_matrix = tfidf.fit_transform(movies['combined_features'].str.lower())
    
    # Calcular a similaridade cosseno
    cosine_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)
    return cosine_sim


# Criar o modelo de similaridade
cosine_sim = create_similarity_model()

ValueError: np.nan is an invalid document, expected byte or unicode string.

In [ ]:
def recommend_movies(input_text):
    input_movie = input_text.lower()
    movies['combined_features'] = movies['combined_features'].str.lower()
    
    try:
        # Pega o índice do primeiro filme correspondente ao input do usuário
        idx = movies[movies['combined_features'].str.contains(input_movie)].index[0]
    except IndexError:
        messagebox.showerror("Erro", "Filme ou gênero não encontrado")
        return []
    
    # Obter os índices dos filmes mais similares
    sim_scores = list(enumerate(cosine_sim[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    
    # Pega os 5 filmes mais similares (exceto o próprio)
    movie_indices = [i[0] for i in sim_scores[1:6]]
    
    # Retorna os títulos dos filmes recomendados
    recommended_movies = movies.iloc[movie_indices]['title'].values
    return recommended_movies

In [9]:
# funcao para buscar

def search_genre():
    genre = genre_entry.get()
    recommended = recommend_movies(genre)
    if recommended:
        messagebox.showinfo("Recomendações", "\n".join(recommended))

In [ ]:

def login():
    email = email_entry.get()
    senha = senha_entry.get()
    
    if email and senha:
        login_frame.pack_forget()
        show_catalog_screen()
    else:
        messagebox.showerror("Erro", "Senha e ou email incorretos")
        
def show_catalog_screen():
    catalog_frame.pack(fill='both', expand=True)
    catalog_label.pack(pady=20)
    genre_entry.pack(pady=5)
    genre_button.pack(pady=10)

root = tk.Tk()
root.title("Movie Changes - Sua plataforma de recomendaçãod e filmes")
root.geometry("400x300")

login_frame = tk.Frame(root)
login_frame.pack(fill='both', expand=True)


tk.Label(login_frame, text="Email:").pack(pady=10)
email_entry = tk.Entry(login_frame)
email_entry.pack()

tk.Label(login_frame, text="Senha:").pack(pady=10)
senha_entry = tk.Entry(login_frame, show='*')
senha_entry.pack()

login_button = tk.Button(login_frame, text="Login", command=login)
login_button.pack(pady=20)

sigin_button = tk.Button(login_frame, text="SigIn", command=login)
sigin_button.pack(pady=20)

catalog_frame = tk.Frame(root)
catalog_label = tk.Label(catalog_frame, text="Qual genero te agrada hoje?")
genre_entry = tk.Entry(catalog_frame)
genre_button = tk.Button(catalog_frame, text="Buscar")



root.mainloop()
